In [4]:
import pandas as pd
import numpy as np
import os as os
import GalanovModel as gm
from scipy.optimize import fsolve


In [5]:
datasets_dir=os.getenv("DATASETS_DIR")

In [6]:
dataset=pd.read_csv(f'{datasets_dir}/prediction_RF.csv', header=0,names=["Zr","Nb",'Mo',"Ti","Cr", "RF - Phase prediction from EBSD class","RF - Phase prediction from XRD class","RF - CI prediction","RF - IQ prediction",	"RF - Young Modulus prediction (GPa)","RF - Hardness prediction (GPa)"])
display(dataset)

# Create index 
nb_data=dataset.shape[0]
index_list=[]
for i in range (0,nb_data):
    index_list.append("compo_"+str(i))


dataset['index']=index_list
dataset=dataset.set_index('index')

dataset_mechanical_model=dataset.copy()

# Add columns that are going to be computed
dataset_mechanical_model['C']=np.zeros(nb_data)
dataset_mechanical_model['x']=np.zeros(nb_data)
dataset_mechanical_model['deltaH']=np.zeros(nb_data)
dataset_mechanical_model['E2/H']=np.zeros(nb_data)

Hs_column="RF - Hardness prediction (GPa)"
Es_column="RF - Young Modulus prediction (GPa)"


,Zr,Nb,Mo,Ti,Cr,RF - Phase prediction from EBSD class,RF - Phase prediction from XRD class,RF - CI prediction,RF - IQ prediction,RF - Young Modulus prediction (GPa),RF - Hardness prediction (GPa)
0,0.0,0.0,0.0,0.0,100.0,1.0,1,0.227018,57893.368438,267.790417,11.316550
1,0.0,0.0,0.0,2.0,98.0,1.0,1,0.226734,57893.368438,236.930690,11.509728
2,0.0,0.0,0.0,4.0,96.0,1.0,1,0.226929,59560.721745,235.365696,11.496725
3,0.0,0.0,0.0,6.0,94.0,1.0,1,0.228720,58880.630167,234.382639,11.411974
4,0.0,0.0,0.0,8.0,92.0,1.0,1,0.232064,59234.431667,241.943527,11.410432
...,...,...,...,...,...,...,...,...,...,...,...
316246,98.0,0.0,0.0,0.0,2.0,0.0,1,0.019261,36816.220090,126.529137,7.808660
316247,98.0,0.0,0.0,2.0,0.0,0.0,1,0.019261,36816.220090,126.529137,7.808660
316248,98.0,0.0,2.0,0.0,0.0,0.0,1,0.019261,36816.220090,126.529137,7.808660
316249,98.0,2.0,0.0,0.0,0.0,0.0,1,0.019261,36816.220090,126.529137,7.808660


In [11]:
nu_s=0.3
nu_i=0.07
E_i=1141

for j in range (0,nb_data):   
    H_s=dataset[Hs_column]['compo_'+str(j)]
    E_s=dataset[Es_column]['compo_'+str(j)]

    [Ei_star,Ks,alpha_s,beta_s,theta_s,cot_gamma_i,z]=gm.Galanov_math_values(E_s,nu_s,E_i,nu_i,H_s)

    # starting points for system solving
    starting_x_y=[[1,1],[2,0.5],[3,0.3],[5,0.25]]

    #initialize list of solution obtained from each starting points
    x_sol_list=[]
    y_sol_list=[]
    
    # for each starting point
    for i in range (0,4):
        # solve solution
        [x_sol, y_sol] = fsolve(gm.system_x_y, starting_x_y[i],args=(alpha_s,beta_s,theta_s,z))

       # Check that the computed values verify the system and that they are in range of Galanov given values
        if np.isclose(gm.system_x_y([x_sol,y_sol],alpha_s,beta_s,theta_s,z), [0.0, 0.0]).all()==True and 1/y_sol<5 and 1/y_sol>0.5 and x_sol<7 and x_sol > 0.5:
            x_sol_list.append(x_sol)
            y_sol_list.append(y_sol)
    
   # if no solution was found
    if len(x_sol_list)==0:
        C=0
        x=0
    else:
        # if the solutions found from the different strating points are equals
        if np.std(x_sol_list)<10**(-8) and np.std(y_sol_list)<10**(-8) :
            C=1/np.mean(y_sol_list)
            x=np.mean(x_sol_list)
        else:
            print('no unique solution')
            
    
    delta_H=gm.delta_H_value(E_s,H_s,nu_s,z)

    dataset_mechanical_model['C']['compo_'+str(j)]=C
    dataset_mechanical_model['x']['compo_'+str(j)]=x
    dataset_mechanical_model['deltaH']['compo_'+str(j)]=delta_H
    dataset_mechanical_model['E2/H']['compo_'+str(j)]=E_s**2/H_s
    

/Users/elisegarel/Desktop/PUBLI/Fichier git/accelerated-exploration-of-multinary/GalanovModel/GalanovModel.py:106: RuntimeWarning: invalid value encountered in log
  eq2=1/y-(2/3+y*(x**3-alpha_s)/beta_s*np.log((1-y*(x**3-alpha_s)/(2*beta_s))**(-1))+2*np.log(x))
/Users/elisegarel/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/Users/elisegarel/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/elisegarel/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [12]:
display(dataset_mechanical_model)
ref=pd.read_csv(f'{datasets_dir}Predictions_RF_mechanical_model.csv',header=0,names=["Zr","Nb",'Mo',"Ti","Cr", "RF - Phase prediction from EBSD class","RF - Phase prediction from XRD class","RF - CI prediction","RF - IQ prediction",	"RF - Young Modulus prediction (GPa)","RF - Hardness prediction (GPa)","C","x","deltaH","E2/H"])
display(ref)
display(dataset_mechanical_model-ref)

,Zr,Nb,Mo,Ti,Cr,RF - Phase prediction from EBSD class,RF - Phase prediction from XRD class,RF - CI prediction,RF - IQ prediction,RF - Young Modulus prediction (GPa),RF - Hardness prediction (GPa),C,x,deltaH,E2/H
index,,,,,,,,,,,,,,,
compo_0,0.0,0.0,0.0,0.0,100.0,1.0,1,0.227018,57893.368438,267.790417,11.316550,1.565623,1.515234,0.716603,6336.887682
compo_1,0.0,0.0,0.0,2.0,98.0,1.0,1,0.226734,57893.368438,236.930690,11.509728,1.437825,1.420911,0.687068,4877.278595
compo_2,0.0,0.0,0.0,4.0,96.0,1.0,1,0.226929,59560.721745,235.365696,11.496725,1.432924,1.417398,0.685912,4818.503452
compo_3,0.0,0.0,0.0,6.0,94.0,1.0,1,0.228720,58880.630167,234.382639,11.411974,1.436346,1.419799,0.686780,4813.822932
compo_4,0.0,0.0,0.0,8.0,92.0,1.0,1,0.232064,59234.431667,241.943527,11.410432,1.465265,1.440639,0.693601,5130.101034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
compo_316246,98.0,0.0,0.0,0.0,2.0,0.0,1,0.019261,36816.220090,126.529137,7.808660,1.246121,1.287817,0.641626,2050.239505
compo_316247,98.0,0.0,0.0,2.0,0.0,0.0,1,0.019261,36816.220090,126.529137,7.808660,1.246121,1.287817,0.641626,2050.239505
compo_316248,98.0,0.0,2.0,0.0,0.0,0.0,1,0.019261,36816.220090,126.529137,7.808660,1.246121,1.287817,0.641626,2050.239505


,Zr,Nb,Mo,Ti,Cr,RF - Phase prediction from EBSD class,RF - Phase prediction from XRD class,RF - CI prediction,RF - IQ prediction,RF - Young Modulus prediction (GPa),RF - Hardness prediction (GPa),C,x,deltaH,E2/H
compo_0,0.0,0.0,0.0,0.0,100.0,1.0,1,0.227018,57893.368438,267.790417,11.316550,1.565623,1.515234,0.716603,6336.887682
compo_1,0.0,0.0,0.0,2.0,98.0,1.0,1,0.226734,57893.368438,236.930690,11.509728,1.437825,1.420911,0.687068,4877.278595
compo_2,0.0,0.0,0.0,4.0,96.0,1.0,1,0.226929,59560.721745,235.365696,11.496725,1.432924,1.417398,0.685912,4818.503452
compo_3,0.0,0.0,0.0,6.0,94.0,1.0,1,0.228720,58880.630167,234.382639,11.411974,1.436346,1.419799,0.686780,4813.822932
compo_4,0.0,0.0,0.0,8.0,92.0,1.0,1,0.232064,59234.431667,241.943527,11.410432,1.465265,1.440639,0.693601,5130.101034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
compo_316246,98.0,0.0,0.0,0.0,2.0,0.0,1,0.019261,36816.220090,126.529137,7.808660,1.246121,1.287817,0.641626,2050.239505
compo_316247,98.0,0.0,0.0,2.0,0.0,0.0,1,0.019261,36816.220090,126.529137,7.808660,1.246121,1.287817,0.641626,2050.239505
compo_316248,98.0,0.0,2.0,0.0,0.0,0.0,1,0.019261,36816.220090,126.529137,7.808660,1.246121,1.287817,0.641626,2050.239505
compo_316249,98.0,2.0,0.0,0.0,0.0,0.0,1,0.019261,36816.220090,126.529137,7.808660,1.246121,1.287817,0.641626,2050.239505


,Zr,Nb,Mo,Ti,Cr,RF - Phase prediction from EBSD class,RF - Phase prediction from XRD class,RF - CI prediction,RF - IQ prediction,RF - Young Modulus prediction (GPa),RF - Hardness prediction (GPa),C,x,deltaH,E2/H
index,,,,,,,,,,,,,,,
compo_0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.000000e+00,0.0,0.0,-2.220446e-16,0.0,0.0,0.0
compo_1,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.000000e+00,0.0,0.0,-4.440892e-16,0.0,0.0,0.0
compo_2,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.000000e+00,0.0,0.0,-2.220446e-16,0.0,0.0,0.0
compo_3,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,-7.275958e-12,0.0,0.0,0.000000e+00,0.0,0.0,0.0
compo_4,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
compo_316246,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.0
compo_316247,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.0
compo_316248,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0,0.0


In [ ]:
 
    starting_x_y=[[1,1],[2,0.5],[3,0.3],[5,0.25]]
    x_sol_list=[]
    y_sol_list=[]




    
    
    x=0
    C=0
    for i in range (0,4):
        [x_sol, y_sol] = fsolve(systeme_x_y, starting_x_y[i])
        print(np.isclose(systeme_x_y([x_sol,y_sol]), [0.0, 0.0]))
        if np.isclose(systeme_x_y([x_sol,y_sol]), [0.0, 0.0]).all()==True and 1/y_sol<5 and 1/y_sol>0.5 and x_sol<7 and x_sol > 0.5:
            x_sol_list.append(x_sol)
            y_sol_list.append(y_sol)
    
    #    print('x_sol_list=',x_sol_list)
    #    print('y_sol_list=',y_sol_list)
    if len(x_sol_list)==0:
        C=0
        x=0
    else:
        if np.std(x_sol_list)<10**(-8) and np.std(y_sol_list)<10**(-8) :
            C=1/np.mean(y_sol_list)
            x=np.mean(x_sol_list)
        else:
            print('no unique solution')
            
    eps_p=-np.log(np.sqrt(1+ z**2))
    eps_l=-(1+nu_s)*(1-2*nu_s)*H_s/E_s
    delta_H=eps_p/(eps_p+eps_l)
    database_mechanical_model['C']['compo_'+str(j)]=C
    database_mechanical_model['x']['compo_'+str(j)]=x
    database_mechanical_model['deltaH']['compo_'+str(j)]=delta_H
    database_mechanical_model['E2/H']['compo_'+str(j)]=E_s**2/H_s
    
x=np.linspace(0.7,6)
C_x=2/3+0.0688+2*np.log(x)
plt.plot(database_mechanical_model['x'],database_mechanical_model['C'],'+',x,C_x)
plt.show()



#%%
# Desirabilité
database_desirabilite=database_mechanical_model[["Zr","Nb","Mo","Ti","Cr","SVM - Phase prediction from EBSD class","SVM - Phase prediction from XRD class","SVM - CI prediction",	"SVM - Hardness prediction (GPa)","deltaH"]]
database_desirabilite["SVM - Hardness prediction (GPa)"]=(database_desirabilite["SVM - Hardness prediction (GPa)"]-database_desirabilite["SVM - Hardness prediction (GPa)"].min())/(database_desirabilite["SVM - Hardness prediction (GPa)"].max()-database_desirabilite["SVM - Hardness prediction (GPa)"].min())
database_desirabilite['deltaH']=(database_desirabilite['deltaH']-database_desirabilite['deltaH'].min())/(database_desirabilite['deltaH'].max()-database_desirabilite['deltaH'].min())
database_desirabilite['indice']=np.sqrt(database_desirabilite["SVM - Hardness prediction (GPa)"]*database_desirabilite['deltaH'])
database_mechanical_model['indice']=np.sqrt(database_desirabilite["SVM - Hardness prediction (GPa)"]*database_desirabilite['deltaH'])

#database_desirabilite.to_csv('./Databases_v6/Predictions_SVM_desirabilite.csv')
database_mechanical_model.to_csv('./Databases_v6/Predictions_SVM_mechanical_model.csv')